In [3]:
def get_class_from_name(name):
    """
    Gets the class from the file name.
    
    The class is defined by the last number written in the file name.
    For example:
    
    Input: ".../1_1_1.mat"
    Output: 1.0
    
    Input: ".../1_1_0.mat"
    Output: 0.0
    """
    try:
        return float(name[-5])
    except:
        return 0.0

assert get_class_from_name('/train_1/1_1_0.mat') == 0.0
assert get_class_from_name('/train_1/1_1_1.mat') == 1.0

In [4]:
from scipy.fftpack import rfft


def get_X_files_and_y(base_dir, train_samples=600):
    ignored_files = ['.DS_Store', '1_45_1.mat']

    X_files = np.array([])
    y = np.array([])

    for i, filename in enumerate(os.listdir(base_dir)):
        if filename in ignored_files:
            continue

        X_files = np.append(X_files, str(filename))
        y = np.append(y, get_class_from_name(filename))  # The number of readings

        if i >= train_samples:
            break
    
    return X_files, y


def get_X_from_files(base_dir, files, show_progress=True):
    """
    Given a list of filenames, returns the Standard deviation of the content of each file as a row.
    """
    X = None
    n = 128

    total_files = len(files)

    for i, filename in enumerate(files):
        if show_progress and i % int(total_files / 10) == 0:
            print(u'%{}: Loading file {}'.format(int(i * 100 / total_files), filename))

        try:
            mat_data = scipy.io.loadmat(''.join([base_dir, filename]))
        except ValueError as ex:
            print(u'Error loading MAT file {}: {}'.format(filename, str(ex)))
            continue

        data = mat_data['dataStruct'][0][0][0]
        x_fft = rfft(data, n=n, axis=0)

        X = np.vstack([X, x_fft]) if X is not None else x_fft
    
    return X

In [36]:
import os
import numpy as np
from sklearn.cross_validation import train_test_split

base = r'C:\data\seizure\train_1'
base_tests = r'C:\data\seizure\test_1'
    
X_files, y = get_X_files_and_y(base, train_samples=250)
X_train_files, X_test_files, y_train, y_test = train_test_split(X_files, y, test_size=0.33, random_state=42)

In [39]:
print(len(X_files))
print(len(y))

251
251


In [13]:
base_dir = 'c:\\data\\seizure\\train1'
for i, filename in enumerate(os.listdir(base_dir)):
    print(fileName)

WindowsError: [Error 3] The system cannot find the path specified: 'c:\\data\\seizure\\train1/*.*'

In [18]:
os.listdir('./seizure/train1')

WindowsError: [Error 3] The system cannot find the path specified: './seizure/train1/*.*'

In [21]:
import os
startpath = "c:data\seizure\test_1"
corpus_path = sorted([os.path.join("c:", "data", directories) for directories in os.listdir(startpath)])
print(corpus_path)

WindowsError: [Error 123] The filename, directory name, or volume label syntax is incorrect: 'c:data\\seizure\test_1/*.*'

In [28]:
from os import listdir
from os.path import isfile, join
mypath = r"C:\data\seizure\test_1"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['1_1.mat', '1_10.mat', '1_100.mat', '1_1000.mat', '1_1001.mat', '1_1002.mat', '1_1003.mat', '1_1004.mat', '1_1005.mat', '1_1006.mat', '1_1007.mat', '1_1008.mat', '1_1009.mat', '1_101.mat', '1_1010.mat', '1_1011.mat', '1_1012.mat', '1_1013.mat', '1_1014.mat', '1_1015.mat', '1_1016.mat', '1_1017.mat', '1_1018.mat', '1_1019.mat', '1_102.mat', '1_1020.mat', '1_1021.mat', '1_1022.mat', '1_1023.mat', '1_1024.mat', '1_1025.mat', '1_1026.mat', '1_1027.mat', '1_1028.mat', '1_1029.mat', '1_103.mat', '1_1030.mat', '1_1031.mat', '1_1032.mat', '1_1033.mat', '1_1034.mat', '1_1035.mat', '1_1036.mat', '1_1037.mat', '1_1038.mat', '1_1039.mat', '1_104.mat', '1_1040.mat', '1_1041.mat', '1_1042.mat', '1_1043.mat', '1_1044.mat', '1_1045.mat', '1_1046.mat', '1_1047.mat', '1_1048.mat', '1_1049.mat', '1_105.mat', '1_1050.mat', '1_1051.mat', '1_1052.mat', '1_1053.mat', '1_1054.mat', '1_1055.mat', '1_1056.mat', '1_1057.mat', '1_1058.mat', '1_1059.mat', '1_106.mat', '1_1060.mat', '1_1061.mat', '1_1062.mat', '1_